### Setup the installs

In [ ]:
!pip install openai
!pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 12.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 14.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993224 sha256=1aad2f8e6eb913df6e5edece16e4ed9f3f1f7e5138781ec8e4f8c434e9312a21
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully

### Import libraries and get the dataframe

In [ ]:
import pandas as pd
from tqdm import tqdm
from langdetect import detect

In [ ]:
df = pd.read_csv('dataset.csv')

### Rephrase Sentence Function

In [ ]:
import openai

# Set your OpenAI API key
openai.api_key = ''

def rephrase_sentence(sentence, full_language):
    """
    Rephrases a given sentence in the specified language using OpenAI API.

    Args:
        sentence (str): The sentence to be rephrased.
        full_language (str): The full name or description of the language.

    Returns:
        str: The rephrased sentence.
    """
    prompt = f"Rephrase the following {full_language} sentence:\n\n" + sentence + f"\n\nRephrased {full_language} sentence:"
    response = openai.Completion.create(
        engine="davinci",
        prompt=prompt,
        max_tokens=len(sentence),
        temperature=0.8,
        n=1,
        stop=None
    )
    rephrased_sentence = response.choices[0].text.strip().split('\n')[0]
    return rephrased_sentence

# Example usage
# sentence = "माझं नाव आहे सोहिनी."
# rephrased = rephrase_sentence(sentence, "Marathi")
# print(f"Original Sentence: {sentence}")
# print(f"Rephrased Sentence: {rephrased}")

In [ ]:
'''
Detect if the text is in the input lang
'''

def is_lang(text,input_lang):
    try:
        lang = detect(text)
        if lang == input_lang:
            return True  # Detected language is Marathi
        else:
            return False  # Detected language is not Marathi
    except:
        return False  # Failed to detect language

### Calling it all together in the main function

In [ ]:
def call_rephrase(df, lang_token, full_language):
    augmented = []
    for index, row in df.iterrows():
        print(index)
        # Access the values of each column for the current row
        sentence = row['text']
        label = row['label']

        # Loop until a rephrased sentence in the desired language is obtained
        while True:
            rephrased = rephrase_sentence(sentence, full_language)
            if is_lang(rephrased, lang_token):
                break  # Break out of the loop if the condition is met

        # Remove the language tag from the rephrased sentence if present
        if f"Rephrased {full_language} sentence:" in rephrased:
            rephrased = rephrased.replace(f"Rephrased {full_language} sentence:", "")

        augmented.append([rephrased, label])

    new_df = pd.DataFrame(augmented, columns=df.columns)
    df_new = df.append(new_df, ignore_index=True)
    df_new = df_new.sample(frac=1)
    df_new.to_csv('df_new.csv')

In [ ]:
call_rephrase(df_test, 'mr','Marathi')

0
1
2
3
4


<ipython-input-7-79065d2670db>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_new = df.append(new_df, ignore_index=True)
